In [7]:
import numpy as np

In [108]:
#T-product (note that in python the number of frontal slices are saved 
#under index 0 and not under index 3
def linear_transform(X, transform):
    L = transform['L']
    n1 = transform['l']
    return np.matmul(L(X, axis=0), transform['inverseL'](np.eye(n1), axis=0))

def inverse_linear_transform(X, transform):
    inverseL = transform['inverseL']
    n1 = transform['l']
    return np.matmul(inverseL(X, axis=0), inverseL(np.eye(n1), axis=0))

def tprod(A, B):
    n1, n2, n3 = A.shape
    m1, m2, m3 = B.shape

    if n3 != m2 or n1 != m1:
        raise ValueError('Inner tensor dimensions must agree.')


    transform = {
        'L': np.fft.fft,
        'l': n1,
        'inverseL': np.fft.ifft
    }

    C = np.zeros((n1, n2, m3), dtype=complex)

    if transform['L'] == np.fft.fft:
        # efficient computing for fft transform
        A = np.fft.fft(A, axis=0)
        B = np.fft.fft(B, axis=0)
        halfn1 = np.ceil((n3 + 1) / 2).astype(int)
        for i in range(halfn1):
            C[i, :, :] = np.matmul(A[i, :, :], B[i, :, :])
        for i in range(halfn1 + 1, n1):
            C[i, :, :] = np.conj(C[n1 + 1 - i, :, :])
        C = np.fft.ifft(C, axis=0)
    else:
        # other transform
        A = linear_transform(A, transform)
        B = linear_transform(B, transform)
        for i in range(n1):
            C[i, :, :] = np.matmul(A[i, :, :], B[i, :, :])
        C = inverse_linear_transform(C, transform)

    return C

In [111]:
#Example 
#it is taken from the paper 
#https://www.researchgate.net/publication/265330442_Third-Order_Tensors_as_Operators_on_Matrices_A_Theoretical_and_Computational_Framework_with_Applications_in_Imaging)

A = np.array([[[1, 0], [0, 2], [-1, 3]], [[-2, 1], [-2, 7], [0, -1]]])
B = np.array([[[3], [-1]], [[-2], [-3]]])

C = tprod(A,B)

print(C)
print(np.shape(C))

[[[  4.+0.j]
  [-19.+0.j]
  [ -3.+0.j]]

 [[ -9.+0.j]
  [-19.+0.j]
  [ -6.+0.j]]]
(2, 3, 1)
